In [64]:
#imports
import pandas as pd
import numpy as np
import string
import time
import higher

#plotting
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

#PyTorch imports
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
# from torchtext.data import Field, TabularDataset, BucketIterator
from torch.autograd import Variable
from sklearn.metrics import f1_score
# import torchtext
import os
import torch.optim as optim

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, GRU


from sklearn.metrics import f1_score, accuracy_score

In [12]:
df = pd.read_csv("../../../Dataset/ADBE_Stock.csv")

df


,Date,Symbol,Adj Close,Close,High,Low,Open,Volume
0,1/2/2019,ADBE,224.570007,224.570007,226.169998,219.000000,219.910004,2784100
1,1/3/2019,ADBE,215.699997,215.699997,223.630005,215.149994,220.880005,3663500
2,1/4/2019,ADBE,226.190002,226.190002,227.649994,217.479996,219.839996,4043400
3,1/7/2019,ADBE,229.259995,229.259995,232.600006,227.289993,229.949997,3638500
4,1/8/2019,ADBE,232.679993,232.679993,233.770004,228.330002,232.649994,3685900
...,...,...,...,...,...,...,...,...
1311,3/19/2024,ADBE,521.190002,521.190002,522.679993,508.989990,509.890015,7081800
1312,3/20/2024,ADBE,519.140015,519.140015,523.869995,512.500000,523.289978,4239600
1313,3/21/2024,ADBE,511.250000,511.250000,519.729981,506.200012,517.599976,5206600
1314,3/22/2024,ADBE,499.519989,499.519989,511.589996,496.670013,509.070007,5410300


In [13]:
df1=df.reset_index()['Close']
df1

0       224.570007
1       215.699997
2       226.190002
3       229.259995
4       232.679993
           ...    
1311    521.190002
1312    519.140015
1313    511.250000
1314    499.519989
1315    507.230011
Name: Close, Length: 1316, dtype: float64

In [17]:
scaler=MinMaxScaler(feature_range=(0,1))
df1=scaler.fit_transform(np.array(df1).reshape(-1,1))
df1

array([[0.01876576],
       [0.        ],
       [0.02219309],
       ...,
       [0.62527769],
       [0.6004612 ],
       [0.61677283]])

In [18]:
# 4. Chia train test
train_size = int(0.7 * len(df1))
test_size = int(0.2 * len(df1))
val_size = len(df1) - train_size - test_size

train_data = df1[:train_size]
test_data = df1[train_size:train_size+test_size]
val_data = df1[train_size+test_size:]

In [20]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, time_step=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-time_step-1):
		a = dataset[i:(i+time_step), 0]   ###i=0, X=0,1,2,3-----99   Y=100 
		dataX.append(a)
		dataY.append(dataset[i + time_step, 0])
	return np.array(dataX), np.array(dataY)

In [21]:
time_step = 100
X_train, y_train = create_dataset(train_data, time_step)
X_val, yval = create_dataset(val_data, time_step)
X_test, ytest = create_dataset(test_data, time_step)

In [22]:
# 7. Reshape input to be [samples, time steps, features] which is required for 
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)
X_val = X_val.reshape(X_val.shape[0],X_val.shape[1] , 1)

In [62]:
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(ytest, dtype=torch.float32)

C:\Users\tuleh\AppData\Local\Temp\ipykernel_8588\2200389102.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train, dtype=torch.float32)
C:\Users\tuleh\AppData\Local\Temp\ipykernel_8588\2200389102.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.tensor(y_train, dtype=torch.float32)
C:\Users\tuleh\AppData\Local\Temp\ipykernel_8588\2200389102.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test, dtype=torch.float32)


In [86]:
class StockPredictor(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(StockPredictor, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        h, _ = self.lstm(x)
        out = self.fc(h[:, -1, :])
        return out

input_size = 1  # Số lượng đặc trưng đầu vào
hidden_size = 64
output_size = 1
learning_rate = 0.01
num_epochs = 10
inner_steps = 5

model = StockPredictor(input_size, hidden_size, output_size)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

# Chuyển đổi dữ liệu thành tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

C:\Users\tuleh\AppData\Local\Temp\ipykernel_8588\935544513.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
C:\Users\tuleh\AppData\Local\Temp\ipykernel_8588\935544513.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
C:\Users\tuleh\AppData\Local\Temp\ipykernel_8588\935544513.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
C:\Users\tuleh\App

In [87]:
class MAML:
    def __init__(self, model, inner_lr, outer_lr, inner_steps):
        self.model = model
        self.inner_lr = inner_lr
        self.outer_lr = outer_lr
        self.inner_steps = inner_steps
        self.meta_optimizer = optim.Adam(self.model.parameters(), lr=outer_lr)
    
    def inner_update(self, x_train, y_train):
        # Clone model parameters
        adapted_params = {name: param.clone() for name, param in self.model.named_parameters()}
        
        # Inner loop
        for _ in range(self.inner_steps):
            predictions = self.model(x_train)
            loss = nn.MSELoss()(predictions, y_train)
            grads = torch.autograd.grad(loss, self.model.parameters(), create_graph=True)
            adapted_params = {name: param - self.inner_lr * grad for (name, param), grad in zip(adapted_params.items(), grads)}
        
        return adapted_params
    
    def meta_update(self, x_train, y_train, x_test, y_test):
        adapted_params = self.inner_update(x_train, y_train)
        
        # Outer loop
        test_predictions = self.model(x_test)
        test_loss = nn.MSELoss()(test_predictions, y_test)
        
        # Meta optimization
        self.meta_optimizer.zero_grad()
        test_loss.backward()
        self.meta_optimizer.step()

    def train(self, x_train_list, y_train_list, x_test_list, y_test_list):
        for x_train, y_train, x_test, y_test in zip(x_train_list, y_train_list, x_test_list, y_test_list):
            self.meta_update(x_train, y_train, x_test, y_test)

# Training loop
input_size = 1   # Số lượng features
hidden_size = 10
output_size = 1
model = StockPredictor(input_size, hidden_size, output_size)
maml = MAML(model, inner_lr=0.01, outer_lr=0.001, inner_steps=1)

# Chuyển đổi thành danh sách tensors để sử dụng trong MAML
x_train_list = [torch.tensor(x, dtype=torch.float32) for x in X_train]
y_train_list = [torch.tensor(y, dtype=torch.float32) for y in y_train]
x_test_list = [torch.tensor(x, dtype=torch.float32) for x in X_test]
y_test_list = [torch.tensor(y, dtype=torch.float32) for y in y_test]

maml.train(x_train_list, y_train_list, x_test_list, y_test_list)

C:\Users\tuleh\AppData\Local\Temp\ipykernel_8588\734704579.py:46: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_train_list = [torch.tensor(x, dtype=torch.float32) for x in X_train]
C:\Users\tuleh\AppData\Local\Temp\ipykernel_8588\734704579.py:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train_list = [torch.tensor(y, dtype=torch.float32) for y in y_train]
C:\Users\tuleh\AppData\Local\Temp\ipykernel_8588\734704579.py:48: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_test_list = [torch.tensor(x, dtype=torch.float32) for x in X_tes

IndexError: too many indices for tensor of dimension 2

In [92]:
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.optim import Adam

# 1. Tiền xử lý dữ liệu
def create_time_series(data, window_size):
    series = []
    for i in range(len(data) - window_size):
        series.append(data[i:i + window_size + 1])
    return np.array(series)

# 2. Xây dựng mô hình MAML
class StockPredictor(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(StockPredictor, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        h, _ = self.lstm(x)
        h = h[:, -1, :]
        return self.fc(h)

def train_maml(model, data, epochs, meta_lr, task_lr, meta_batch_size):
    optimizer = Adam(model.parameters(), lr=meta_lr)
    loss_fn = nn.MSELoss()

    for epoch in range(epochs):
        meta_loss = 0
        for _ in range(meta_batch_size):
            task_data = data[np.random.choice(len(data), meta_batch_size)]
            task_model = StockPredictor(input_size, hidden_size).to(device)
            task_model.load_state_dict(model.state_dict())
            task_optimizer = Adam(task_model.parameters(), lr=task_lr)

            for _ in range(task_epochs):
                for sequence in task_data:
                    x, y = sequence[:-1], sequence[-1]
                    x = torch.tensor(x, dtype=torch.float32).to(device).unsqueeze(-1)  # Thêm chiều cho đầu vào LSTM
                    y = torch.tensor(y, dtype=torch.float32).to(device)
                    y_pred = task_model(x.unsqueeze(0))
                    task_loss = loss_fn(y_pred, y.unsqueeze(0))
                    task_optimizer.zero_grad()
                    task_loss.backward()
                    task_optimizer.step()

            meta_loss += task_loss.item()

        optimizer.zero_grad()
        meta_loss_tensor = torch.tensor(meta_loss, requires_grad=True)  # Đảm bảo meta_loss là tensor
        meta_loss_tensor.backward()
        optimizer.step()
        print(f'Epoch {epoch + 1}, Meta Loss: {meta_loss / meta_batch_size}')

# 3. Đánh giá mô hình
def evaluate(model, data):
    model.eval()
    predictions, targets = [], []
    with torch.no_grad():
        for sequence in data:
            x, y = sequence[:-1], sequence[-1]
            x = torch.tensor(x, dtype=torch.float32).to(device).unsqueeze(-1)  # Thêm chiều cho đầu vào LSTM
            y = torch.tensor(y, dtype=torch.float32).to(device)
            y_pred = model(x.unsqueeze(0))
            predictions.append(y_pred.item())
            targets.append(y.item())
    return predictions, targets

# Main script
if __name__ == "__main__":
    # Giả sử bạn đã có dữ liệu trong DataFrame
    # Hãy thay thế `your_data` bằng DataFrame của bạn
    your_data = pd.read_csv('../../../Dataset/ADBE_Stock.csv')
    close_prices = your_data['Close'].values

    window_size = 20
    series = create_time_series(close_prices, window_size)

    input_size = 1
    hidden_size = 50
    task_epochs = 5
    meta_lr = 0.001
    task_lr = 0.01
    meta_batch_size = 16
    epochs = 100

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = StockPredictor(input_size, hidden_size).to(device)

    train_maml(model, series, epochs, meta_lr, task_lr, meta_batch_size)

    # Evaluate the model
    predictions, targets = evaluate(model, series[-100:])
    print(f'Predictions: {predictions}')
    print(f'Targets: {targets}')


c:\Program Files\Python311\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1, Meta Loss: 154369.63037109375
Epoch 2, Meta Loss: 200709.34521484375
Epoch 3, Meta Loss: 169081.9453125
Epoch 4, Meta Loss: 167268.32299804688
Epoch 5, Meta Loss: 171969.3359375
Epoch 6, Meta Loss: 166254.6005859375
Epoch 7, Meta Loss: 155000.42138671875
Epoch 8, Meta Loss: 253423.7470703125
Epoch 9, Meta Loss: 171204.455078125
Epoch 10, Meta Loss: 149157.10473632812
Epoch 11, Meta Loss: 187365.56396484375
Epoch 12, Meta Loss: 188601.49658203125
Epoch 13, Meta Loss: 203994.4296875
Epoch 14, Meta Loss: 177572.41625976562
Epoch 15, Meta Loss: 218623.72827148438
Epoch 16, Meta Loss: 179691.41162109375
Epoch 17, Meta Loss: 193367.2822265625
Epoch 18, Meta Loss: 200440.75830078125
Epoch 19, Meta Loss: 171624.32250976562
Epoch 20, Meta Loss: 172451.3154296875
Epoch 21, Meta Loss: 224770.54736328125
Epoch 22, Meta Loss: 162193.18920898438
Epoch 23, Meta Loss: 197079.46923828125
Epoch 24, Meta Loss: 225348.51708984375
Epoch 25, Meta Loss: 133157.04467773438
Epoch 26, Meta Loss: 171928

In [101]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

# Đọc dữ liệu từ file CSV
def load_data(file_path):
    data = pd.read_csv(file_path)
    return data['Close'].values

# Tạo các tasks
def create_tasks(data, task_size, num_tasks):
    tasks = []
    for _ in range(num_tasks):
        start_idx = np.random.randint(0, len(data) - task_size)
        task_data = data[start_idx:start_idx + task_size]
        tasks.append(task_data)
    return tasks

# Định nghĩa mô hình
def build_model(input_shape):
    model = tf.keras.Sequential([
        layers.Dense(64, activation='relu', input_shape=input_shape),
        layers.Dense(64, activation='relu'),
        layers.Dense(1)
    ])
    return model

# Chuẩn bị dữ liệu cho từng task
def prepare_task_data(tasks, look_back=1):
    prepared_tasks = []
    for task in tasks:
        X, y = [], []
        for i in range(len(task) - look_back):
            X.append(task[i:i + look_back])
            y.append(task[i + look_back])
        X = np.array(X).reshape(-1, look_back)
        y = np.array(y).reshape(-1, 1)
        split_index = int(len(X) * 0.8)
        X_train, y_train = X[:split_index], y[:split_index]
        X_val, y_val = X[split_index:], y[split_index:]
        prepared_tasks.append({'X_train': X_train, 'y_train': y_train, 'X_val': X_val, 'y_val': y_val})
    return prepared_tasks

# Áp dụng MAML
class MAML:
    def __init__(self, model, meta_lr, task_lr):
        self.model = model
        self.meta_optimizer = tf.keras.optimizers.Adam(meta_lr)
        self.task_lr = task_lr

    def adapt(self, task_data):
        with tf.GradientTape() as tape:
            predictions = self.model(task_data['X_train'], training=True)
            loss = tf.keras.losses.MSE(task_data['y_train'], predictions)
        gradients = tape.gradient(loss, self.model.trainable_variables)
        k_shot_model = build_model(task_data['X_train'].shape[1:])
        k_shot_model.set_weights(self.model.get_weights())
        k_shot_optimizer = tf.keras.optimizers.Adam(self.task_lr)
        k_shot_optimizer.apply_gradients(zip(gradients, k_shot_model.trainable_variables))
        return k_shot_model

    def meta_train(self, tasks, num_epochs):
        for epoch in range(num_epochs):
            all_gradients = []
            for task_data in tasks:
                k_shot_model = self.adapt(task_data)
                with tf.GradientTape() as tape:
                    predictions = k_shot_model(task_data['X_val'], training=False)
                    loss = tf.keras.losses.MSE(task_data['y_val'], predictions)
                gradients = tape.gradient(loss, self.model.trainable_variables)
                all_gradients.append(gradients)
            mean_gradients = [tf.reduce_mean(gradients, axis=0) for gradients in zip(*all_gradients)]
            self.meta_optimizer.apply_gradients(zip(mean_gradients, self.model.trainable_variables))

    def predict(self, x, k_shot_model=None):
        if k_shot_model is None:
            k_shot_model = self.model
        return k_shot_model.predict(x)

# Đánh giá mô hình
def evaluate_model(maml, test_data):
    k_shot_model = maml.adapt(test_data)
    predictions = maml.predict(test_data['X_val'], k_shot_model)
    return predictions

# Main code
file_path = '../../../Dataset/ADBE_Stock.csv'  # Đường dẫn tới file CSV
data = load_data(file_path)

# Thiết lập các tham số
task_size = 50
num_tasks = 10
look_back = 1  # Số bước lùi
input_shape = (look_back,)  # Đảm bảo hình dạng đúng
meta_lr = 0.001
task_lr = 0.01
num_epochs = 100

# Tạo các tasks
tasks = create_tasks(data, task_size, num_tasks)

# Chuẩn bị dữ liệu cho từng task
prepared_tasks = prepare_task_data(tasks, look_back)

# Xây dựng và huấn luyện mô hình MAML
model = build_model(input_shape)  # Đảm bảo hình dạng đầu vào đúng
maml = MAML(model, meta_lr, task_lr)
maml.meta_train(prepared_tasks, num_epochs)

# Đánh giá mô hình
test_task = prepared_tasks[0]  # Sử dụng một task từ tập dữ liệu đã chuẩn bị để kiểm tra
predictions = evaluate_model(maml, test_task)
print(predictions)


ValueError: Attempt to convert a value (None) with an unsupported type (<class 'NoneType'>) to a Tensor.